In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

23/03/06 10:32:21 WARN Utils: Your hostname, honeyeater resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
23/03/06 10:32:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/06 10:32:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version

'3.3.2'

# Question 2

In [5]:
from pyspark.sql import types

In [11]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [15]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('week5/raw_data/fhvhv_tripdata_2021-06.csv')

In [16]:
df = df.repartition(12)

In [17]:
df.write.parquet('week5/parquet')

In [19]:
!ls -lh week5/parquet

total 271M
-rw-r--r-- 1 apeel apeel 23M Mar  6 11:07 part-00000-742f2140-abca-453f-9a1e-0c35020f034f-c000.snappy.parquet
-rw-r--r-- 1 apeel apeel 23M Mar  6 11:07 part-00001-742f2140-abca-453f-9a1e-0c35020f034f-c000.snappy.parquet
-rw-r--r-- 1 apeel apeel 23M Mar  6 11:07 part-00002-742f2140-abca-453f-9a1e-0c35020f034f-c000.snappy.parquet
-rw-r--r-- 1 apeel apeel 23M Mar  6 11:07 part-00003-742f2140-abca-453f-9a1e-0c35020f034f-c000.snappy.parquet
-rw-r--r-- 1 apeel apeel 23M Mar  6 11:07 part-00004-742f2140-abca-453f-9a1e-0c35020f034f-c000.snappy.parquet
-rw-r--r-- 1 apeel apeel 23M Mar  6 11:07 part-00005-742f2140-abca-453f-9a1e-0c35020f034f-c000.snappy.parquet
-rw-r--r-- 1 apeel apeel 23M Mar  6 11:07 part-00006-742f2140-abca-453f-9a1e-0c35020f034f-c000.snappy.parquet
-rw-r--r-- 1 apeel apeel 23M Mar  6 11:07 part-00007-742f2140-abca-453f-9a1e-0c35020f034f-c000.snappy.parquet
-rw-r--r-- 1 apeel apeel 23M Mar  6 11:07 part-00008-742f2140-abca-453f-9a1e-0c35020f034f-c000.snappy.parquet

# Question 3

In [21]:
df = spark.read.parquet('week5/parquet')

In [22]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [23]:
from pyspark.sql import functions as F

In [24]:
df.count()

14961892

In [26]:
from datetime import date

In [29]:
df.filter(F.to_date(df.pickup_datetime) == date(2021, 6, 15)).show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02879|2021-06-15 08:47:23|2021-06-15 08:59:10|         239|         230|      N|                B02879|
|              B02872|2021-06-15 17:31:11|2021-06-15 17:48:23|         195|         144|      N|                B02872|
|              B02764|2021-06-15 14:35:27|2021-06-15 14:57:48|          36|          61|      N|                B02764|
|              B02510|2021-06-15 17:25:28|2021-06-15 17:32:13|         220|         220|      N|                  null|
|              B02880|2021-06-15 15:59:55|2021-06-15 16:51:05|          10|         265|      N|                B02880|
|              B02869|2021-06-15 19:45:4

In [28]:
df.filter(F.to_date(df.pickup_datetime) == date(2021, 6, 15)).count()

452470

# Question 4

In [30]:
df \
  .withColumn('trip_duration', df.dropoff_datetime - df.pickup_datetime) \
  .show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|       trip_duration|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|              B02617|2021-06-04 16:55:38|2021-06-04 17:01:55|         143|         142|      N|                B02617|INTERVAL '0 00:06...|
|              B02877|2021-06-04 03:18:39|2021-06-04 03:29:53|          80|         211|      N|                B02877|INTERVAL '0 00:11...|
|              B02883|2021-06-04 11:56:31|2021-06-04 12:15:25|          61|          17|      N|                B02883|INTERVAL '0 00:18...|
|              B02764|2021-06-02 22:53:33|2021-06-02 23:04:04|          68|         265|      N|                B02764|INTERVAL '0 00:10...|
|            

In [33]:
longest_trip = df \
  .withColumn('trip_duration', df.dropoff_datetime - df.pickup_datetime) \
  .agg(F.max('trip_duration')) \
  .collect()

In [34]:
longest_trip

[Row(max(trip_duration)=datetime.timedelta(days=2, seconds=67964))]

In [46]:
hours = longest_trip[0]['max(trip_duration)'].total_seconds() / (60 * 60)

In [47]:
hours

66.8788888888889

# Question 6

In [48]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('data/taxi_zone_lookup.csv')

In [51]:
df_zones.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [52]:
zones_schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [53]:
df_zones = spark.read \
    .option("header", "true") \
    .schema(zones_schema) \
    .csv('data/taxi_zone_lookup.csv')

In [54]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [55]:
df_with_zones = df.join(df_zones, df.PULocationID == df_zones.LocationID)

In [56]:
df_with_zones.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+---------+--------------------+------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|LocationID|  Borough|                Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+---------+--------------------+------------+
|              B02617|2021-06-04 16:55:38|2021-06-04 17:01:55|         143|         142|      N|                B02617|       143|Manhattan| Lincoln Square West| Yellow Zone|
|              B02877|2021-06-04 03:18:39|2021-06-04 03:29:53|          80|         211|      N|                B02877|        80| Brooklyn|   East Williamsburg|   Boro Zone|
|              B02883|2021-06-04 11:56:31|2021-06-04 12:15:25|          61|          17|      N|                B02883|      

In [58]:
df_with_zones.groupBy('Zone').count().show()

+--------------------+------+
|                Zone| count|
+--------------------+------+
|           Homecrest| 42555|
|              Corona| 48578|
|    Bensonhurst West| 52137|
|         Westerleigh| 10980|
|Charleston/Totten...|  5481|
|          Douglaston|  8849|
|      Newark Airport|   228|
|          Mount Hope| 73909|
|East Concourse/Co...| 74920|
|      Pelham Parkway| 50198|
|         Marble Hill| 14781|
|           Rego Park| 38686|
|Upper East Side S...|124621|
|       Dyker Heights| 24657|
|Heartland Village...| 15737|
|   Kew Gardens Hills| 24636|
|       Rikers Island|     3|
|             Bayside| 28515|
|     Jackson Heights|114413|
|TriBeCa/Civic Center|164344|
+--------------------+------+
only showing top 20 rows



In [61]:
df_with_zones.groupBy('Zone').count().orderBy('count', ascending=False).show()

+--------------------+------+
|                Zone| count|
+--------------------+------+
| Crown Heights North|231279|
|        East Village|221244|
|         JFK Airport|188867|
|      Bushwick South|187929|
|       East New York|186780|
|TriBeCa/Civic Center|164344|
|   LaGuardia Airport|161596|
|            Union Sq|158937|
|        West Village|154698|
|             Astoria|152493|
|     Lower East Side|151020|
|        East Chelsea|147673|
|Central Harlem North|146402|
|Williamsburg (Nor...|143683|
|          Park Slope|143594|
|  Stuyvesant Heights|141427|
|        Clinton East|139611|
|West Chelsea/Huds...|139431|
|             Bedford|138428|
|         Murray Hill|137879|
+--------------------+------+
only showing top 20 rows

